In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from os.path import abspath

# SparkSession
URL_SPARK = "spark://spark-master:7077"
warehouse_location = abspath('spark-warehouse')

spark = (
    SparkSession.builder
    .appName("read-stream-from-stored-stream-minio")
    .config("executor.memory", "8g")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("spark.jars", "jars/hadoop-aws-3.3.2.jar,jars/aws-java-sdk-bundle-1.12.172.jar,jars/hadoop-common-3.3.2.jar,jars/spark-avro_2.12-3.3.2.jar,jars/commons-pool2-2.11.1.jar")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .master(URL_SPARK)
    .getOrCreate()
)

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/10/08 19:42:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/08 19:42:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark.sql('show databases')

namespace
default


23/10/08 19:53:01 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/10/08 19:53:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://sparkjupyter-minio-1:9000"

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://sparkjupyter-minio-1:9000")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minioadmin")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minioadmin" )
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")

In [ ]:
path_to_stored_minio = "s3a://external-acumos-vm-data-transformed/"

In [ ]:
df_stream = spark.readStream.text(path_to_stored_minio)

In [ ]:
# df_stream.writeStream.format('console').start()